In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["MKL_NUM_THREADS"] = "4"
os.environ["VECLIB_MAXIMUM_THREADS"] = "4"
os.environ["NUMEXPR_NUM_THREADS"] = "4"

import pandas as pd
from pathlib import Path
import prody as pr
import multiprocess

def get_sequence(path, name='laser'):
    structure = pr.parsePDB(str(path))
    return {f'{name}_path': path, 'sequence': structure.ca.getSequence()}


all_paths = [pdb for pdb in Path('./carpdock_out/laser_outputs/').glob('*/*.pdb')]

all_data = []
get_laser_sequence = lambda path: get_sequence(path, name='laser')
with multiprocess.Pool(128) as p:
    for result in p.imap_unordered(get_laser_sequence, all_paths):
        all_data.append(result)


In [2]:
import pandas as pd
df = pd.DataFrame(all_data)
ligand_smiles = 'COC1=CC=C(N2C3=C(C(C(N)=O)=N2)CCN(C4=CC=C(N5CCCCC5=O)C=C4)C3=O)C=C1'

In [3]:
df

laser_path  \
0      carpdock_out/laser_outputs/cluster_44_sup-clus...   
1      carpdock_out/laser_outputs/cluster_32_sup-clus...   
2      carpdock_out/laser_outputs/cluster_44_sup-clus...   
3      carpdock_out/laser_outputs/cluster_39_sup-clus...   
4      carpdock_out/laser_outputs/cluster_00_sup-clus...   
...                                                  ...   
14995  carpdock_out/laser_outputs/cluster_03_sup-clus...   
14996  carpdock_out/laser_outputs/cluster_03_sup-clus...   
14997  carpdock_out/laser_outputs/cluster_03_sup-clus...   
14998  carpdock_out/laser_outputs/cluster_14_sup-clus...   
14999  carpdock_out/laser_outputs/cluster_06_sup-clus...   

                                                sequence  
0      ATEEDIAAVKKLVKEFIEAMNNRDLDKILEMLDPNARALTGNKDFE...  
1      DKEADIQAVKEWVKELVEAVNSGDFEKLKSLGAEDCKYINLDTGKT...  
2      ATEEDIAAVKKLVRDFIEALNNKDLDKIVEMLDPNARALTGNKDLE...  
3      MTEEEAKAEVKELGEKAVKAYEEGDRALLEEILADDFVFVDKDTGE...  
4      SPIVEEIKKIGEEGLAAMDNGDTEAVLSLAADDAVVTDLDINKTYT...  
...                                                  ...  
14995  MSVEEQVKKLVQTMFDALNAGDAETLRSLLAPDAVGTASDGTVRTT...  
14996  MSVEEQVRELGETAWRAGNAGDAATLRSLAAPDGVLTLDDGTVLRT...  
14997  MSDEAKVRKLGQTAMDAGNAGDAATLRSLAAPDGVLTMSDGRVFRT...  
14998  MDEEELRARARELVRRALALANAGDTAKLDELLAADFRHHVSDGTV...  
14999  LTDEEVRRGVEEFVAAINAGDLDALRRMADPDAVGVLPDGTVLRSR...  

[15000 rows x 2 columns]

In [4]:
all_paths = sorted([pdb for pdb in Path('./boltz_results_boltz_inputs/').glob('predictions/*/*.pdb')])

all_data = []
with multiprocess.Pool(128) as p:
    get_boltz_sequence = lambda path: get_sequence(path, name='boltz')
    for result in p.imap_unordered(get_boltz_sequence, all_paths):
        all_data.append(result)

df = pd.merge(df, pd.DataFrame(all_data), on='sequence')
df

laser_path  \
0      carpdock_out/laser_outputs/cluster_24_sup-clus...   
1      carpdock_out/laser_outputs/cluster_00_sup-clus...   
2      carpdock_out/laser_outputs/cluster_35_sup-clus...   
3      carpdock_out/laser_outputs/cluster_44_sup-clus...   
4      carpdock_out/laser_outputs/cluster_35_sup-clus...   
...                                                  ...   
14995  carpdock_out/laser_outputs/cluster_03_sup-clus...   
14996  carpdock_out/laser_outputs/cluster_05_sup-clus...   
14997  carpdock_out/laser_outputs/cluster_46_sup-clus...   
14998  carpdock_out/laser_outputs/cluster_03_sup-clus...   
14999  carpdock_out/laser_outputs/cluster_03_sup-clus...   

                                                sequence  \
0      MSEEEGRELGAALMAAFNAKDLEAIRRIAGEDAQLHDEDTNTTYTT...   
1      SPIVEEIKKIGEEGLAAMDNGDTEAVLSLAADDAVVTDLDINKTYT...   
2      NELVKLVKEIFDEIHTKKNGDAIRKYFAPDAKAYVKNKNKTYDLDE...   
3      ATEEDIAAVKKLVKEFIEAMNNRDLDKILEMLDPNARALTGNKDFE...   
4      NELVKKVKEIFKEIHTKKNGDALRKYMAPDAKAHVVNDNKTYDLEE...   
...                                                  ...   
14995  MSDEAKVRKLGQTAMDAGNAGDAATLRSLAAPDGVLTMSDGRVFRT...   
14996  SAEEKKAAVKALGAAAVAAMNAGDIDALRAIVHPEARLTDKNTGKT...   
14997  LTKAEKTAAATALIKEFLASCDAGDIDRVRSLLAEDATVTAKGTNK...   
14998  MSVEEQVRELGETAWRAGNAGDAATLRSLAAPDGVLTLDDGTVLRT...   
14999  MSVEAQVRKLGETAWEAGNAGDAETLRALAAPDGVLTLSDGRVFTT...   

                                              boltz_path  
0      boltz_results_boltz_inputs/predictions/0/0_mod...  
1      boltz_results_boltz_inputs/predictions/1/1_mod...  
2      boltz_results_boltz_inputs/predictions/11/11_m...  
3      boltz_results_boltz_inputs/predictions/2/2_mod...  
4      boltz_results_boltz_inputs/predictions/54/54_m...  
...                                                  ...  
14995  boltz_results_boltz_inputs/predictions/14999/1...  
14996  boltz_results_boltz_inputs/predictions/14996/1...  
14997  boltz_results_boltz_inputs/predictions/14942/1...  
14998  boltz_results_boltz_inputs/predictions/14993/1...  
14999  boltz_results_boltz_inputs/predictions/14943/1...  

[15000 rows x 3 columns]

In [5]:
import json
from calc_symmetry_aware_rmsd import main as calc_rmsd

def get_metadata(laser_path, boltz_path, smiles):
    protein_rmsd, ligand_rmsd, ligand_atom_name_mapping = calc_rmsd(laser_path, boltz_path, smiles, align_on_binding_site=True)

    boltz_confidence_json_path = boltz_path.parent / f'confidence_{boltz_path.stem}.json'
    with open(boltz_confidence_json_path, 'r') as f:
        boltz_confidence_data = json.load(f)

    struct = pr.parsePDB(str(boltz_path))
    protein_plddt = struct.select('chid A and name CA').getBetas().mean()
    ligand_plddt = struct.select('chid B').getBetas().mean()
    
    return {
        'boltz_path': boltz_path,
        'protein_rmsd': protein_rmsd, 'ligand_rmsd': ligand_rmsd, 
        'ligand_atom_name_mapping': ligand_atom_name_mapping, 
        'iptm': boltz_confidence_data['iptm'], 'ligand_plddt': ligand_plddt, 
        'protein_plddt': protein_plddt
    }

all_data = []
all_inputs = [(row['laser_path'], row['boltz_path'], ligand_smiles) for idx, row in df.iterrows()]
with multiprocess.Pool(128) as p:
    for result in p.starmap(get_metadata, all_inputs):
        all_data.append(result)


In [6]:
df_ = pd.merge(df, pd.DataFrame(all_data), on='boltz_path')
df_filtered = df_[(df_.protein_rmsd < 2.0) & (df_.ligand_rmsd < 2.0)].sort_values(by='ligand_rmsd').copy()

In [7]:
import freesasa
freesasa.setVerbosity(freesasa.silent)

carboxamide_boltz_atom_names = ['N49', 'C31', 'O26']

def calc_carboxamide_and_full_ligand_sasa(boltz_path):
    strct = freesasa.Structure(str(boltz_path), options={'hetatm': True})
    result = freesasa.calc(strct)
    selection = freesasa.selectArea(['carboxamide, chain B and (name N49 or name C31 or name O26)', 'total_lig, chain B'], strct, result)
    return {'boltz_path': boltz_path, 'total_sasa': selection['total_lig'], 'carboxamide_sasa': selection['carboxamide']}

all_data = []
all_inputs = [row['boltz_path'] for idx, row in df_filtered.iterrows()]
with multiprocess.Pool(128) as p:
    for result in p.imap_unordered(calc_carboxamide_and_full_ligand_sasa, all_inputs):
        all_data.append(result)

df_final = pd.merge(df_filtered, pd.DataFrame(all_data), on='boltz_path')
df_final


laser_path  \
0     carpdock_out/laser_outputs/cluster_20_sup-clus...   
1     carpdock_out/laser_outputs/cluster_06_sup-clus...   
2     carpdock_out/laser_outputs/cluster_20_sup-clus...   
3     carpdock_out/laser_outputs/cluster_39_sup-clus...   
4     carpdock_out/laser_outputs/cluster_12_sup-clus...   
...                                                 ...   
2955  carpdock_out/laser_outputs/cluster_47_sup-clus...   
2956  carpdock_out/laser_outputs/cluster_07_sup-clus...   
2957  carpdock_out/laser_outputs/cluster_25_sup-clus...   
2958  carpdock_out/laser_outputs/cluster_39_sup-clus...   
2959  carpdock_out/laser_outputs/cluster_17_sup-clus...   

                                               sequence  \
0     ATAAEMVKRVLESLQAFEEGDTEKLRAMAAPDARLVAVATGEELHS...   
1     MTEREVREFGAEFLRAVNEGDLDALRRMLDPDAVLVLPDGRRLNSR...   
2     ATAEEMVKRVKEALKAFEEGDTEKLKSLANPDAKLVAVETGETLNS...   
3     MSDEEAKKEVEEIIKKAIAAFESGDRETLEEIMADDFVMTDLDTGK...   
4     LTAEEVRQRCVEFVEAFNAGDKEAIKKFAAPGATLHLADGTVLTTL...   
...                                                 ...   
2955  MTVEEKKQATIDIIQAAVDALNAGDLEALRSLLAEDAVVELGDGTV...   
2956  MTDEEIREAVKRLIAAIETGDIERIRELCAPDCKVEWPDGSTSNSI...   
2957  EKSLKEKCEELVKEIIDAFNAGDVEKLLSLAAPDATAEGSDGTKYT...   
2958  MTDEEAKKAVEEICAKLGAAVEKGDKETLKEILADDFVHIDKDTGE...   
2959  GGEEEIKKLIKELVDAFNNKDIEKLKELLGEDAKLVDKTTNVSLNK...   

                                             boltz_path  protein_rmsd  \
0     boltz_results_boltz_inputs/predictions/3567/35...      0.536329   
1     boltz_results_boltz_inputs/predictions/3291/32...      0.595210   
2     boltz_results_boltz_inputs/predictions/3566/35...      0.649612   
3     boltz_results_boltz_inputs/predictions/7849/78...      0.510658   
4     boltz_results_boltz_inputs/predictions/1845/18...      0.742989   
...                                                 ...           ...   
2955  boltz_results_boltz_inputs/predictions/4262/42...      1.669494   
2956  boltz_results_boltz_inputs/predictions/7969/79...      1.389572   
2957  boltz_results_boltz_inputs/predictions/10036/1...      0.976554   
2958  boltz_results_boltz_inputs/predictions/8676/86...      0.501295   
2959  boltz_results_boltz_inputs/predictions/13321/1...      1.232372   

      ligand_rmsd                           ligand_atom_name_mapping  \
0        0.470353  {'C15': 'C53', 'O4': 'O30', 'C1': 'C43', 'C2':...   
1        0.486864  {'C15': 'C53', 'O4': 'O30', 'C1': 'C43', 'C2':...   
2        0.504021  {'C15': 'C53', 'O4': 'O30', 'C1': 'C43', 'C6':...   
3        0.534109  {'C15': 'C53', 'O4': 'O30', 'C1': 'C43', 'C6':...   
4        0.536738  {'C15': 'C53', 'O4': 'O30', 'C1': 'C43', 'C6':...   
...           ...                                                ...   
2955     1.996755  {'C15': 'C53', 'O4': 'O30', 'C1': 'C43', 'C2':...   
2956     1.997949  {'C15': 'C53', 'O4': 'O30', 'C1': 'C43', 'C2':...   
2957     1.998366  {'C15': 'C53', 'O4': 'O30', 'C1': 'C43', 'C6':...   
2958     1.998547  {'C15': 'C53', 'O4': 'O30', 'C1': 'C43', 'C2':...   
2959     1.999118  {'C15': 'C53', 'O4': 'O30', 'C1': 'C43', 'C6':...   

          iptm  ligand_plddt  protein_plddt  total_sasa  carboxamide_sasa  
0     0.968844     94.725588      96.497250   72.310167          8.109211  
1     0.807672     56.762059      94.739492   25.252417         17.340924  
2     0.961687     94.803824      97.058750   59.999306          4.684365  
3     0.956475     92.155294      97.960159   33.183273          5.894366  
4     0.888039     54.260294      89.164017   16.655459          9.933439  
...        ...           ...            ...         ...               ...  
2955  0.752289     46.896765      92.721371   13.123138          0.000000  
2956  0.842703     74.942647      92.092881   67.664210         44.027513  
2957  0.824000     75.476765      96.061220   38.036745          9.291476  
2958  0.856596     62.492941      94.707381   53.020305         24.880675  
2959  0.897063     78.290588      93.594750   68.

In [13]:
print(df_final[df_final.boltz_path == (df_final.boltz_path.iloc[0].parent.parent / '6330' / '6330_model_0.pdb')].iloc[0].laser_path.absolute())

df_final[df_final.boltz_path == (df_final.boltz_path.iloc[0].parent.parent / '6330' / '6330_model_0.pdb')]

# df_final.boltz_path.iloc[0].parent.parent.absolute(), df_final.boltz_path.iloc[0]

/nfs/polizzi/bfry/programs/CARPdock/apixaban_ntf2_screen/carpdock_out/laser_outputs/cluster_25_sup-cluster_95_94/design_0.pdb


laser_path  \
1086  carpdock_out/laser_outputs/cluster_25_sup-clus...   

                                               sequence  \
1086  APSLEEQCRALGEAFVAAANAGDVEAMLSMLDPDATMELSDGTKLT...   

                                             boltz_path  protein_rmsd  \
1086  boltz_results_boltz_inputs/predictions/6330/63...      0.916851   

      ligand_rmsd                           ligand_atom_name_mapping  \
1086     1.359368  {'C15': 'C53', 'O4': 'O30', 'C1': 'C43', 'C2':...   

          iptm  ligand_plddt  protein_plddt  total_sasa  carboxamide_sasa  
1086  0.966586     93.428529      96.431301   26.554012         17.229723

In [22]:
import shutil
opath = Path('./selected_poses/')
opath.mkdir(exist_ok=True, parents=True)
selected_poses = df_final[(df_final['carboxamide_sasa'] > 10) & (df_final['total_sasa'] < df_final['total_sasa'].median()) & (df_final['ligand_plddt'] > 80)].sort_values('ligand_plddt', ascending=False).copy()

for idx, row in selected_poses.iterrows():
    boltz_path = row['boltz_path']
    outpath = opath / boltz_path.name
    shutil.copy(boltz_path, outpath)